Baigus darbą visu failus suarchyvuoti ir atsiųsti email tomas.rasymas@gmail.com su savo vardu ir pavarde.

Kodą būtina komentuoti.

# 1 užduotis

Sukurti modelius:

**Shop (parduotuvė)**, lentelės pavadinimas - shops:
* id - sveikas skaičius, pirminis raktas
* name - tekstas 40 simbolių ilgio, privalomas (nėra tuščias)
* address - tekstas 100 simbolių ilgio
* items - ryšys į Item modelį

**Item (prekė)**, lentelės pavadinimas - items:
* id - sveikas skaičius, pirminis raktas
* barcode - tekstas 32 simbolių ilgio, unikalus
* name - tekstas 40 simbolių ilgio, privalomas (nėra tuščias)
* description - tekstas 200 simbolių ilgio, numatyta reikšmė - EMPTY
* unit_price - dešimtainis skaičius su tikslumu (10, 2), privalomas (nėra tuščias), numatyta reikšmė 1.00
* created_at - data ir laikas (DateTime), numatyta reikšmė - įrašo sukūrimo data ir laikas
* shop_id - sveikas skaičius, ForeignKey į shops.id
* shop - ryšys į Shop modelį
* components - ryšys į Component modelį

**Component (komponentas)**, lentelės pavadinimas - components:
* id - sveikas skaičius, pirminis raktas
* name - tekstas 20 simbolių ilgio
* quantity - dešimtainis skaičius su tikslumu (10, 2), numatyta reikšmė 1.00
* item_id - sveikas skaičius, ForeignKey į items.id
* item - ryšys į Item modelį

Viena parduotuvė (shop) turi daug prekių (item), viena prekė (item) yra sudaryta iš daug komponentų.

# 2 užduotis

Sukurti įrašus modeliuose:

**Shop:**
* name='IKI', address='Kaunas, Iki gatvė 1'
* name='MAXIMA', address='Kaunas, Maksima gatvė 2'

**Item:** 
* **IKI shop**
  * barcode='112233112233', name='Žemaičių duona', unit_price=1.55
  * barcode='33333222111', description='Pienas iš Žemaitijos', name='Žemaičių pienas', unit_price=2.69
* **MAXIMA shop**
  * barcode='99898989898', name='Aukštaičių duona', unit_price=1.65
  * barcode='99919191991', description='Pienas iš Aukštaitijos', name='Aukštaičių pienas', unit_price=2.99
 
**Component**
* **IKI Žemaičių duona**
  * name='Miltai', quantity=1.50
  * name='Vanduo', quantity=1.00
* **IKI Žemaičių pienas**
  * name='Pienas', quantity=1.00
* **MAXIMA Aukštaičių duona**
  * name='Miltai', quantity=1.60
  * name='Vanduo', quantity=1.10
* **MAXIMA Aukštaičių pienas**
  * name='Pienas', quantity=1.10

# 3 užduotis

* Pakeisti 'IKI' parduotuvės, 'Žemaičių duonos' komponento 'Vandens' kiekį (quantity) iš 1.00 į 1.45.
* Ištrinti 'MAXIMA' parduotuvės, 'Aukštaičių pieno' komponentą 'Pienas'.

# 4 užduotis

Atspausdinti visų parduotuvių prekes, bei tų prekių komponentus.

# 5 užduotis

Sukurti užklausas:
* Atrinkti prekes, kurios turi susietų komponentų
* Atrinkti prekes, kurių pavadinime yra tekstas 'ien'
* Suskaičiuoti iš kiek komponentų sudaryta kiekviena prekė
* Suskaičiuoti kiekvienos prekės komponentų kiekį (quantity)
* Savo nuožiūra suformuoti pasirinktų duomenų užklausą, bei ją aprašyti.

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, Numeric, DateTime
from sqlalchemy.orm import relationship
import datetime

engine = create_engine("sqlite:///:memory:", echo=True) 
Base = declarative_base()

class Shop(Base):
    __tablename__ = "shops"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(40), nullable=False)
    address = Column(String(100))
    
    def __repr__(self):
        return "<Shop(id='%s', name='%s', address='%s')>" % (self.id, self.name, self.address)
                     
class Item(Base):
    __tablename__ = "items"
    
    id = Column(Integer, primary_key=True)
    barcode = Column(String(32), unique=True)
    name = Column(String(40), nullable=False)
    description = Column(String(200), default='EMPTY')
    unit_price = Column(Numeric(10, 2), nullable=False, default=1.00)
    created_at = Column(DateTime, default=datetime.datetime.now())
    
    shop_id = Column(Integer, ForeignKey("shops.id"))
    
    def __repr__(self):
        return "<Item(id='%s', name='%s', barcode='%s', description='%s', unit_price='%s', created_at='%s', shop_id='%s')>" % (self.id, self.name, self.barcode, self.description, self.unit_price, self.created_at, self.shop_id)

class Component(Base):
    __tablename__ = 'components'
    id = Column(Integer, primary_key=True)
    name = Column(String(20))
    quantity = Column(Numeric(10, 2), default=10.00)
    
    item_id = Column(Integer, ForeignKey("items.id"))
    
    def __repr__(self):
        return "<Component(id='%s', name='%s', quantity='%s', item_id='%s')>" % (self.id, self.name, self.quantity, self.item_id)

Shop.items = relationship("Item", back_populates="shop")
Item.shop = relationship("Shop", back_populates="items")
Item.components = relationship("Component", back_populates="item")
Component.item = relationship("Item", back_populates="components")
    
Base.metadata.create_all(engine)





from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

iki = Shop(name='IKI', address='Kaunas, Iki gatvė 1')
maxima = Shop(name='MAXIMA', address='Kaunas, Maksima gatvė 2')

session.add_all([iki, 
                 maxima])

session.commit()




iki_duona = Item(barcode='112233112233', name='Žemaičių duona', unit_price=1.55)
iki_pienas = Item(barcode='33333222111', description='Pienas iš Žemaitijos', name='Žemaičių pienas', unit_price=2.69)

maxima_duona = Item(barcode='99898989898', name='Aukštaičių duona', unit_price=1.65)
maxima_pienas = Item(barcode='99919191991', description='Pienas iš Aukštaitijos', name='Aukštaičių pienas', unit_price=2.99)

iki.items = [iki_duona, iki_pienas]
maxima.items = [maxima_duona, maxima_pienas]

iki_duona_miltai = Component(name='Miltai', quantity=1.50)
iki_duona_vanduo = Component(name='Vanduo', quantity=1.00)
iki_pienas_pienas = Component(name='Pienas', quantity=1.00)

iki_duona.components = [iki_duona_miltai, iki_duona_vanduo]
iki_pienas.components = [iki_pienas_pienas]

maxima_duona_miltai = Component(name='Miltai', quantity=1.60)
maxima_duona_vanduo = Component(name='Vanduo', quantity=1.10)
maxima_pienas_pienas = Component(name='Pienas', quantity=1.10)

maxima_duona.components = [maxima_duona_miltai, maxima_duona_vanduo]
maxima_pienas.components = [maxima_pienas_pienas]

session.add_all([iki_duona, 
                 iki_pienas,
                 maxima_duona,
                 maxima_pienas])





iki_duona_vanduo.quantity = 1.45
session.add(iki_duona_vanduo)
session.commit()

session.delete(maxima_pienas_pienas)
session.commit()





for s in session.query(Shop).all():
    print(s)
    for i in s.items:
        print(i)
        for c in i.components:
            print(c)
            
            
            
            
from sqlalchemy.sql import exists

stmt = exists().where(Item.id==Component.item_id)
for i in session.query(Item).filter(stmt):
    print(i)

    
    
    
for i in session.query(Item).filter(Item.name.like("%ien%")).all():
    print(i)

    

    
from sqlalchemy import func

for i, c in session.query(Item, func.count(Component.id)).join(Component).group_by(Item).all():
    print(i, c)
    
    
    
    
    
from sqlalchemy import func

for i, c in session.query(Item, func.sum(Component.quantity)).join(Component).group_by(Item).all():
    print(i, c)

2018-04-30 10:38:08,035 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-30 10:38:08,037 INFO sqlalchemy.engine.base.Engine ()
2018-04-30 10:38:08,039 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-30 10:38:08,041 INFO sqlalchemy.engine.base.Engine ()
2018-04-30 10:38:08,044 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("shops")
2018-04-30 10:38:08,045 INFO sqlalchemy.engine.base.Engine ()
2018-04-30 10:38:08,048 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("items")
2018-04-30 10:38:08,049 INFO sqlalchemy.engine.base.Engine ()
2018-04-30 10:38:08,052 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("components")
2018-04-30 10:38:08,053 INFO sqlalchemy.engine.base.Engine ()
2018-04-30 10:38:08,056 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE shops (
	id INTEGER NOT NULL, 
	name VARCHAR(40) NOT NULL, 
	address VARCHAR(100), 
	PRIMARY KEY (id)
)


2018-04-30

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sqlalchemy/sql/sqltypes.py:603: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


<Item(id='1', name='Žemaičių duona', barcode='112233112233', description='EMPTY', unit_price='1.55', created_at='2018-04-30 10:38:08.025771', shop_id='1')> 2.95
<Item(id='2', name='Žemaičių pienas', barcode='33333222111', description='Pienas iš Žemaitijos', unit_price='2.69', created_at='2018-04-30 10:38:08.025771', shop_id='1')> 1.00
<Item(id='3', name='Aukštaičių duona', barcode='99898989898', description='EMPTY', unit_price='1.65', created_at='2018-04-30 10:38:08.025771', shop_id='2')> 2.70
